In [95]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Psychology and Recommendations
One area I've been intrigued by for a while is recommendation systems. I think they hold a lot of potential for the types of problems we face as psychologists because at their core they tackle a psychological problem, predicting what people like. More than that though the tackle this problem in a way that I think is particularly useful to pscyhological science, and that is with a focus on individual differences. While a lot of psychological research for a long time has sought to draw general conclusions about all people (i.e. anything that uses a group average) but recommendation systems (at least some of them) focus on learning a model that take into account how people differ.

One can clearly see how this might work in one of the classic contexts of recommendations systems, recommending what movies you might like based on your previous preferences. As a side note my understanding of why this is such a classic context is that sometime in 2000s netflix had the netflix competition which was a large cash prize (1 million?) for whoever could do this most effectively, which is just an interesting sociological note on how money implicitly shapes the field. But I digress if you get a little negative and straw persony about psychology you might say that the way most psychologists would approach this problem is to just show a bunch of people movies, find the movies that were rated the highest on average then recommend those movies to every one. This approach in my view essentially posits that there are intrinsic properties of movies that you can use to make recommendations (or "latent factors" if you wanted to be technical about). And you know what this isn't a horrible idea - most good movies people agree are good movies. A slightly more sophisticated approach to this problem might be to stratify this model in some way, for instance you could group people based on age (i.e. a the best movie for a 4 year old might not be the best movie for 65 year old). This approach essentially posits that how much you like a movie is a combination of you as an individual, and the movie.

This approach to say that some psychological experience is both a product of the individual and the context seems like a reasonable starting point to me. With the caveat that context (and to some extent individual) are largely vague terms that could encompass a lot of things (i.e. time of day, weather, time of last meal, time of year, etc.) but for now it seems reasonable to define context as the movie. So after becoming curious about these systems, and feeling like I understood them to some extent conceptually I wanted to get my hands a little dirty coding because concept =/= implementation.

Now just like any area of machine learning it seems like the literature on recommendation systems is pretty vast, but given my interests a simple collaborative filtering approach, which takes the form of matrix decomposition seemed like a reasonable starting point to start playing around. For two reasons 1 it's simple, and 2 it answers the questions I'm interested in, which is what happens when try to model psychological (and other phenomonem) in this way. It's also worth noting that although I haven't done anything with item response theory I do know it takes a similar approach and has a history in psycho-metrics in particular for developing tests of intelligence. I mention item response theory to straw man psychology a little bit less, and to say that there are well established approaches in psychology that take a similar individual + context (item in item response theory) to predict behavior. 

## Recommendation Systems, Affect, and Physiology.

Now one way to go about getting my hands dirty would be to use a classic dataset and implement my own code for something that has been done on one of these classic (probably a movie data set), I went a different way. Since a lot of my interest in these systems comes from what they may or may not be able to tell us about psychological phenomon I used one of the datasets from a lab I work. This data set has ~100 people watching ~30 movies and providing both affective ratings, while we also measure their physiology.

One of the things that's become very clearly to me in my PhD is that physiology =/= affect (or emotion). There's a wealth of scientific literature showing this, but interestingly this scientific knowledge has not filtered through to the general public and people often say something along the lines of physiology is an objective measure of affect or emotion. I shouldn't even disparage the general public you here the same thing all the time at large emotion conferences, and while to be fair many scientists hold this opinion I think the probability of this argument being true is vanishingly small. Yet I digress again. But also not really. Because my point is that now that more and more people have wearables there's this idea that we can dynamically track our emotions across the day or our life, and if we could just do this we could also intervene on these emotions in realtime. Now while I'm a little skeptical about whether this sort of realtime technological intervention would be effective let alone desirable. My thought was to take an area of ML I'm interested in and combine it with some psychological questions I'm interested in (mind-body ~ connection ~), and see how it goes for point out that this relationship is far more complex than is often recognized in the literature. 

A worthy caveat here is that these questions are most often brought up in the context of stress. Although stress is one of those slippery terms that it's not exactly clear what it means to most people, there are some somewhat clear indicators of physiological stress (increases or decreases in heart rate all other things being held equal, which is a more complex thing physiologically then likely any one reading this is actually interested in). But with that being said what these changes in physiology actually mean for the health of an organism (health is another slippery word), I think this is far less clear than we mostly appreciate. I.e. there is good stress and bad stress which physiologically in any one moment may look very similar (or identifcal given the few physiological parameters most wearables measure) so again I'm a little skeptical of this. but again I digress.

## What I actually did 

Below I train a classic collaborative filtering model with probalistic matrix decomposition for 3 different recommendation tasks (or matrix decomposition tasks). One that is trained using continous fear ratings as the target variable given a movie, and person. Then two more that use two different physiological measures (heart rate, and skin conductance responses/s a measure of sympathetic nervous system activty).

I ended up training the models with the FastAI library because before I ran out of time on winter break I had an aspirations of building some more advanced models on top of the based dot product of user and video latent factors + user_bias + video bias collaborative filtering model, but alas it's back to research for me!


## Video HP Losses

The file `video_hp_losses.csv` contains the losses for the video recommendation models. These losses represent the error between the predicted ratings and the actual ratings for each video. The lower the loss, the better the model's performance.

| Video ID | Loss |
|----------|------|
| 1        | 0.05 |
| 2        | 0.10 |
| 3        | 0.08 |
| ...      | ...  |

The table above shows a sample of the video ID and corresponding loss values. Each row represents a video, and the loss value indicates how well the model predicts the fear rating for that video.